In [0]:
import json

bronze_params = dbutils.widgets.get('bronze_params')
print(f"Raw bronze_params: {bronze_params}")

output_data = json.load(bronze_params)


start_date = output_data.get["start_date", ""]
end_date = output_data.get["end_date", ""]
bronze_adls = output_data.get["bronze_adls", ""]
silver_adls = output_data.get["silver_adls", ""]
gold_adls = output_data.get["gold_adls", ""]

print(f"Start Date: {start_date}, Bronze ADLS: {bronze_adls}")

In [0]:
from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta

In [0]:
df = spark.read.option("multiline", "true").json(f"{bronze_adls}{start_date}_earthquake_data.json")

In [0]:
df.head()

Row(geometry=Row(coordinates=[-110.0405, 44.6808333333333, 12.93], type='Point'), id='uu80102611', properties=Row(alert=None, cdi=None, code='80102611', detail='https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=uu80102611&format=geojson', dmin=0.09872, felt=None, gap=94.0, ids=',uu80102611,', mag=1.35, magType='md', mmi=None, net='uu', nst=8, place='36 km S of Silver Gate, Montana', rms=0.15, sig=28, sources=',uu,', status='reviewed', time=1740441234210, title='M 1.4 - 36 km S of Silver Gate, Montana', tsunami=0, type='earthquake', types=',origin,phase-data,', tz=None, updated=1740493416400, url='https://earthquake.usgs.gov/earthquakes/eventpage/uu80102611'), type='Feature')

In [0]:
df

DataFrame[geometry: struct<coordinates:array<double>,type:string>, id: string, properties: struct<alert:string,cdi:double,code:string,detail:string,dmin:double,felt:bigint,gap:double,ids:string,mag:double,magType:string,mmi:double,net:string,nst:bigint,place:string,rms:double,sig:bigint,sources:string,status:string,time:bigint,title:string,tsunami:bigint,type:string,types:string,tz:string,updated:bigint,url:string>, type: string]

In [0]:
# Reshape earthquake data
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)
  

In [0]:
df

DataFrame[id: string, longitude: double, latitude: double, elevation: double, title: string, place_description: string, sig: bigint, mag: double, magType: string, time: bigint, updated: bigint]

In [0]:
df.head()

Row(id='uu80102611', longitude=-110.0405, latitude=44.6808333333333, elevation=12.93, title='M 1.4 - 36 km S of Silver Gate, Montana', place_description='36 km S of Silver Gate, Montana', sig=28, mag=1.35, magType='md', time=1740441234210, updated=1740493416400)

In [0]:

# Validate data: Check for missing or null values
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)
   

In [0]:
# Convert 'time' and 'updated' to timestamp from Unix time
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)

In [0]:
df.head

<bound method DataFrame.head of DataFrame[id: string, longitude: double, latitude: double, elevation: double, title: string, place_description: string, sig: bigint, mag: double, magType: string, time: timestamp, updated: timestamp]>

In [0]:
# Save the transformed DataFrame to the Silver container
silver_output_path = f"{silver_adls}earthquake_events_silver/"

In [0]:
# Append DataFrame to Silver container in Parquet format
df.write.mode('append').parquet(silver_output_path)

In [0]:
dbutils.notebook.exit(silver_output_path)